In [48]:
#!g2.1
!rm -rf toaster-lm
!git clone https://github.com/h1de0us/toaster-lm.git

[Errno 2] No such file or directory: '/home/jupyter/work/resources/toaster-lm/toaster-lm'


Cloning into 'toaster-lm'...


In [21]:
!bash run.sh download_data
!cd .. # return to toaster-lm
!python3 parse_stories.py

4967871


In [7]:
# %pip install -r requirements.txt

In [50]:
#!g2.1
from src.model import TransformerDecoder

from src.dataset import TextDataset

%load_ext autoreload
%autoreload 2

In [54]:
#!g2.1
VOCAB_SIZE = 3000
MAX_LENGTH = 256


text_dataset = TextDataset(data_file="stories.txt",
                           train=True,
                           sp_model_prefix="sp_model",
                           vocab_size=VOCAB_SIZE,
                           max_length=MAX_LENGTH)

In [58]:
#!g2.1
import torch
from src.trainloop import train
from torch.utils.data import DataLoader
from src.utils import collate_fn

BATCH_SIZE = 512
NUM_EPOCHS = 5

train_loader = DataLoader(text_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, pin_memory=True, shuffle=True)
val_loader = DataLoader(text_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, pin_memory=True, shuffle=False)

In [53]:
#!g2.1
import os
os.environ["WANDB_API_KEY"] = "<YOUR_API_KEY>"

import wandb
wandb.login()

wandb.init(project="<YOUR_PROJECT_NAME>")

wandb: Currently logged in as: h1de0us. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.0
wandb: Run data is saved locally in /home/jupyter/work/resources/toaster-lm/wandb/run-20231204_012543-q1dkcnyn
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run sparkling-water-30
wandb: ⭐️ View project at https://wandb.ai/h1de0us/LLM-Homework
wandb: 🚀 View run at https://wandb.ai/h1de0us/LLM-Homework/runs/q1dkcnyn


In [59]:
#!g2.1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TransformerDecoder(vocab_size=text_dataset.vocab_size,
                            embed_dim=768,
                            n_blocks=8,
                            n_head=8,
                            ff_dim=512,
                            text_dataset=text_dataset,
                          use_flash_attention=True)
print(f"Number of parameters in the model: {sum(p.numel() for p in model.parameters())}")
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, betas=(0.9, 0.98), eps=1e-9)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=9500)
criterion = torch.nn.CrossEntropyLoss(ignore_index=text_dataset.pad_id)

Number of parameters in the model: 29836216


In [ ]:
#!g2.1
train_losses, test_losses = train(model=model, optimizer=optimizer, criterion=criterion,
                                  train_loader=train_loader, test_loader=val_loader, 
                                  num_epochs=NUM_EPOCHS,
                                  dataset=text_dataset,
                                  scheduler=scheduler, 
                                  clip_grad=False,
                                  max_gradient_norm=1., 
                                  log_epoch=1)